<img src="https://lh3.googleusercontent.com/pw/ACtC-3fFHZrzKpHGWl0vYz7Sr8FX8QqLQ_tc8XHBSwqQnM4hgsIOjtjaOde1M9oHSAfe1Fs2SwVORlapit4-JOz0mjP8Tnz6HetkLZDZb8CifSd0uoSp1Nj3wG_wh1sEQlKXXzvEA9Y9HnQqu2Ecv2igmInb=w1097-h235-no?authuser=0" alt="2020年度ゲノム情報解析入門" height="100px" align="middle">

# 機械学習 - ゲノム解析 -

　機械学習は、現在のゲノム解析でよく利用されています。

例えば、
- ゲノム全体にわたる遺伝的変異から作物の収穫量を予測するモデルを作成し、育種に活用する（Genomic selection）
- 遺伝子配列（DNA配列やアミノ酸配列）からパターンを読み取り、遺伝子の機能やタンパク質の構造、種類などを予測する（機能予測、構造予測、トポロジー予測など）

　今回、「アミノ酸配列から膜貫通タンパク質かどうかを予測する」分類モデルを作ってみましょう。


## アミノ酸配列から膜貫通タンパク質を予測する

　遺伝子産物であるタンパク質は、アミノ酸で構成されています。タンパク質の種類や機能を知ることは、そのタンパク質（または、タンパク質をコードする遺伝子）の働きを理解するための第一歩になります。

　例えば、下記に代表されるように、アミノ酸配列から[膜タンパク質（Membrane protein）](https://en.wikipedia.org/wiki/Membrane_protein)かどうかを予測するツールがあります。

- [TMHMM](http://www.cbs.dtu.dk/services/TMHMM/)
- [SOSUI](http://harrier.nagahama-i-bio.ac.jp/sosui/sosui_submit.html)

　ここでは、そのようなツールの簡易版を作成していきます。

<small>※ 上述のTMHMMやSOSUIは、与えられたアミノ酸配列が膜タンパク質かどうかの予測だけでなく、膜貫通領域の部位がどこかも予測できます。今回作成する簡易版ツールは、「膜タンパク質かどうかの予測」のみをおこなうツールです。</small>

## 使用するデータセット

[数値化前のデータセット](https://github.com/CropEvol/lecture/blob/master/data/membrane_dataset.csv): 
- 806行 x 2列
  - 列
    - `label`列: 「膜貫通タンパク質」かどうかが書かれた列
    - `sequence`列: アミノ酸配列が書かれた列
  - 行
    - ヘッダー行（1行）: 列名が書かれた行
    - `membrane`ラベル行（105行）: 「膜貫通タンパク質」ラベル 
    - `non_membrane` ラベル行（700行）: 「膜貫通タンパク質でない」ラベル

　アミノ酸配列のようなデータセットを用いる場合、そのままでは機械学習のデータとして利用できません。まず、　アミノ酸配列をなんらかの数値データに変換する必要があります。




　数値化の一例として、ここでは「アミノ酸配列」を「各アミノ酸の割合」に変換しています。このようなデータセットの数値化は、機械学習の工程でいうと、「前処理」に含まれます。</small>

[数値化後のデータセット](https://github.com/CropEvol/lecture/blob/master/data/membrane_dataset.AAfreq.csv): 
- `label`: 目的変数（正解ラベル）
- `A`-`Y`: 説明変数（20個のアミノ酸の割合）。各アミノ酸の種類を一文字表記にしています（詳しくは[こちら](https://ja.wikipedia.org/wiki/%E3%82%BF%E3%83%B3%E3%83%91%E3%82%AF%E8%B3%AA%E3%82%92%E6%A7%8B%E6%88%90%E3%81%99%E3%82%8B%E3%82%A2%E3%83%9F%E3%83%8E%E9%85%B8)）。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/AA_to_dataset.png?raw=true" alt="AminoAcids_to_dataset" height="150px">


<small>※ このデータセットのソースは、TMHMM (Sonnhammer et al., 1998) の[トレーニングデータ](http://www.cbs.dtu.dk/~krogh/TMHMM/)です。  
> Sonnhammer, E. L., Von Heijne, G., & Krogh, A. (1998, July). A hidden Markov model for predicting transmembrane helices in protein sequences. In Ismb (Vol. 6, pp. 175-182).</small>

　次のコードセルを実行して、「数値化後のデータセット」をダウンロードしてください。

In [ ]:
# データセットのダウンロード
!wget -q -O membrane_dataset.AAfreq.csv https://raw.githubusercontent.com/CropEvol/lecture/master/data/membrane_dataset.AAfreq.csv

# 読み込み
import pandas as pd
df = pd.read_csv("membrane_dataset.AAfreq.csv", sep=',', header=0)
df

## 実習内容

 1. 前処理
 2. モデルの作成・学習・評価
 3. 予測

### 前処理

　予測モデルを作成する前に、前処理をおこなう必要があります。ここでは、次の3つの前処理をおこなっています。

1. データの準備
  - 説明変数: 各アミノ酸の割合の列
  - 目的変数: `label`列

1. データの分割
  - トレーニング用のデータとテスト用のデータを用意します。
  - ここでは、トレーニングデータ70%、テストデータ30%に分割。

1. スケーリング
  - 各説明変数の尺度を揃えます。
  - ここでは、各説明変数を正規化（0~1の範囲のデータに変換）。  
  <small> ※ 「アミノ酸の割合」をデータとして扱うため、今回この操作はあまり重要ではありません。</small>

1. ラベルの数値変換
  - ラベル（membrane, non_membrane）を数値データ（0,1）に変換します。


　次のセルを実行して、前処理をおこなってください。


In [ ]:
# 前処理
# (1) データの準備
import numpy as np
x = np.array(df.iloc[:, 1:])  # 説明変数（各アミノ酸の割合）
y = np.array(df["label"])   # 目的変数（ラベル）

# (2) データの分割
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y, random_state=0)

# (3) 説明変数のスケーリング
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
x_train_mm = mms.fit_transform(x_train) # トレーニングデータ
x_test_mm = mms.transform(x_test) # テストデータ

# (4) 目的変数の数値変換
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train_le = le.fit_transform(y_train) # トレーニングデータ
y_test_le = le.transform(y_test) # テストデータ

# 確認
print(y_test_le)

### モデルの作成・学習・評価

　モデルの作成、学習、評価をおこなっていきます。

```python
# モデルの作成
from sklearn.機能 import 関数 
モデル変数 = 関数(オプション)
# モデルの学習
モデル変数.fit(説明変数, 目的変数) # トレーニングデータ
# モデルの評価
モデル変数.score(説明変数, 目的変数) # トレーニングデータ or テストデータ
```

In [ ]:
# 使用するデータ
X_train, Y_train = x_train_mm, y_train_le  # トレーニングデータ
X_test, Y_test  = x_test_mm, y_test_le   # テストデータ

# モデルの作成（ロジスティック回帰）
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
# モデルの学習
clf.fit(X_train, Y_train)

# モデルの評価（正解率）
print("training data: ", clf.score(X_train, Y_train)) # トレーニングデータ
print("test data: ",    clf.score(X_test, Y_test))  # テストデータ

### 予測

　上で作成したモデルを使って、新しいデータのラベル（membrane/non_membrane）を予測します。

予測したい新しいデータ（4つのタンパク質）: 
- [数値化前](https://github.com/CropEvol/lecture/blob/master/data/membrane_newdata.csv)
- [数値化後](https://github.com/CropEvol/lecture/blob/master/data/membrane_newdata.AAfreq.csv)
  - SUT1_ORYSJ: [Sucrose transport protein SUT1 / Oryza sativa](https://www.uniprot.org/uniprot/Q10R54)
  - SCAM1_ORYSJ: [Secretory carrier-associated membrane protein 1 / Oryza sativa](https://www.uniprot.org/uniprot/Q8H5X5)
  - MADS7_ORYSJ: [MADS-box transcription factor 7 / Oryza sativa](https://www.uniprot.org/uniprot/Q0J466)
  - GAOX2_ORYSJ: [Gibberellin 20 oxidase 2](https://www.uniprot.org/uniprot/Q0JH50)

<small>※ それぞれのアミノ酸配列のデータは、タンパク質データベース[UniProt](https://www.uniprot.org/)から得ています。</small>

　次のコードセルを実行して、4つのタンパク質の数値変換後データを取得してください。

In [ ]:
# 新しいデータのダウンロード
!wget -q -O membrane_newdata.AAfreq.csv https://raw.githubusercontent.com/CropEvol/lecture/master/data/membrane_newdata.AAfreq.csv

# 読み込み
new_df = pd.read_csv("membrane_newdata.AAfreq.csv", sep=',', header=0)
new_df

```python
# 予測
モデル変数.predict(新しいデータ)
```

In [ ]:
# 前処理: データの準備と正規化
x_new = np.array(new_df.iloc[:, 1:])
X_new = mms.transform(x_new)

# 予測
y_pred = clf.predict(X_new)
print(le.inverse_transform(y_pred))  # 数値変換前ラベルに戻して表示

### 実習

　「実習コード」のコードセルに追記して、下記の目標をクリアした予測モデルを構築してください。

> 目標: テストデータの正解率 91%以上


　使用する分類手法は自由に選んでください（下記にリンクをクリックすると、scikit-learnのマニュアルページが開きます）。
  - [ロジスティック回帰](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)
  - [サポートベクトルマシン (SVM)](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)
  - [決定木](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)
  - [ランダムフォレスト](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
  - [ニューラルネットワーク](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)


#### 各モデルのコード例

  - [ロジスティック回帰](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)
  ```python
  clf = LogisticRegression() # デフォルト設定（詳しくは、マニュアルページ参照）
  clf = LogisticRegression(solver="liblinear", max_iter=100)  # 最適化手法: Liblinear
  clf = LogisticRegression(solver="newton-cg", max_iter=100)  # 最適化手法: ニュートン-CG法
  clf = LogisticRegression(solver="sag", max_iter=100)  # 最適化手法: Stochastic Average Gradient
  ```
  - [サポートベクトルマシン (SVM)](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)
  ```python
  clf = SVC() # デフォルト設定（詳しくは、マニュアルページ参照）
  clf = SVC(kernel="linear") # 線形SVM
  clf = SVC(kernel="poly") # カーネルSVM (Polynomial)
  clf = SVC(kernel="poly") # カーネルSVM (RBF)
  ```
  - [決定木](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)
  ```python
  clf = DecisionTreeClassifier() # デフォルト設定（詳しくは、マニュアルページ参照）
  
  # 分岐に使う指標: ジニ不純度
  clf = DecisionTreeClassifier(criterion='gini', max_depth=3)  # 枝の深さ: 最大3
  clf = DecisionTreeClassifier(criterion='gini', max_depth=None)  # 枝の深さ: 制限なし
  # 分岐に使う指標: エントロピー（情報量）
  clf = DecisionTreeClassifier(criterion='entropy', max_depth=3)
  clf = DecisionTreeClassifier(criterion='entropy', max_depth=None)
  ```
  - [ランダムフォレスト](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
  ```python
  clf = RandomForestClassifier() # デフォルト設定（詳しくは、マニュアルページ参照）
  clf = RandomForestClassifier(n_estimators=100, max_depth=3)  # 決定木数: 100、枝の深さ: 最大3
  clf = RandomForestClassifier(n_estimators=100, max_depth=None)  # 決定木数: 100、枝の深さ: 上限なし
  ```
  - [ニューラルネットワーク](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)
  ```python
  clf = MLPClassifier()  # デフォルト設定（詳しくは、マニュアルページ参照）

  # 隠れ層: 1層各100ユニット
  # 最適化: Adam (ADAptive Moment estimation)
  # 学習率: 0.001、トレーニング回数: 1000
  clf = MLPClassifier(hidden_layer_sizes=(100,), solver="adam", learning_rate_init=0.001, max_iter=200)

  # 隠れ層: 3層各3ユニット
  # 最適化: 確率的勾配降下法(SGD)
  # 学習率: 0.01、トレーニング回数: 1000
  clf = MLPClassifier(hidden_layer_sizes=(3,3,3,), solver="sgd", learning_rate_init=0.1, max_iter=1000)
  ```

#### 実習コード

In [ ]:
# 使用するデータ
X_train, Y_train = x_train_mm, y_train_le  # トレーニングデータ
X_test, Y_test  = x_test_mm, y_test_le   # テストデータ
X_new  # 新しいデータ

# ===== 編集エリア: start ===== 
# モデルの準備
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
clf = 

# ===== 編集エリア: end  ===== 

# モデルの学習
clf.fit(X_train, Y_train)

# モデルの評価（正解率）
print("training data: ", clf.score(X_train, Y_train)) # トレーニングデータ
print("test data: ",    clf.score(X_test, Y_test))  # テストデータ

# 予測
# 読み込み
new_df = pd.read_csv("membrane_newdata.AAfreq.csv", sep=',', header=0)
x_new = np.array(new_df.iloc[:, 1:])

# 前処理（正規化）
y_pred = clf.predict(X_new)      # 予測値
print(le.inverse_transform(y_pred))  # 予測値を数値変換前ラベルに戻して表示

#### 解答例

　下記の分類手法のいずれかで、目標を達成したモデルを構築できます。
- サポートベクトルマシン
- ランダムフォレスト

In [ ]:
# 使用するデータ
X_train, Y_train = x_train_mm, y_train_le  # トレーニングデータ
X_test, Y_test  = x_test_mm, y_test_le   # テストデータ
X_new  # 新しいデータ

# ===== 編集エリア: start ===== 
# モデルの準備
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
clf = SVC(kernel="rbf")
clf = RandomForestClassifier(n_estimators=100, max_depth=6, random_state=0)

# ===== 編集エリア: end  ===== 

# モデルの学習
clf.fit(X_train, Y_train)

# モデルの評価（正解率）
print("training data: ", clf.score(X_train, Y_train)) # トレーニングデータ
print("test data: ",    clf.score(X_test, Y_test))  # テストデータ

# 予測
y_pred = clf.predict(X_new)      # 予測値
print(le.inverse_transform(y_pred))  # 予測値を数値変換前ラベルに戻して表示

## 良いモデルを構築するために重要なこと

　上述の実習では、どの分類手法を使うかや、パラメータの値をどうするか、に焦点を当てていましたが、機械学習で一番重要な部分は「前処理: データセットの準備」です。

　今回の場合、アミノ酸配列の数値化方法をもう少し工夫をすることで、予測モデルを改善できるかもしれません。一般的に、膜貫通タンパク質には、15-30個の連続した疎水性アミノ酸領域を配列中に保有しています。これは、細胞膜を貫通する領域（膜貫通領域）が、疎水性のアミノ酸で構成されているためです。したがって、このような情報を考慮したデータセットを準備することで、予測モデルの改善が期待できます。

<small>※ 例えば、実習では「個々のアミノ酸の割合」をデータに使いましたが、「連続する複数個のアミノ酸の割合（アミノ酸配列の割合）」を使用することで、予測モデルを改善できるかもしれません。</small>

　**良いデータがなければ、良いモデルはできません。**

---
## まとめ

　次世代シーケンシング技術の発達により、ゲノムワイドなデータを容易に取得できるようになってきました。アクセス可能なゲノムデータの増加に伴い、機械学習を利用したゲノムデータ解析が"あたり前"の時代になってきています。

　ゲノムデータを解析するための重要なポイントは、ゲノムデータを数値化することです。しかし、闇雲に数値化すれば良いわけではありません。「良いモデル」を構築するためには、適切に情報を数値化して、「良いデータセット」を作る必要があります。  
　データセットができれば、ここ数回の実習で見てきたように、自分で機械学習ベースのデータ解析をおこなうことが可能です。

　今後、自身の研究や仕事でデータ解析をおこなうことがあるかと思います。その際には、自分でコーディングして解析することにチャレンジしてみてください。